# **Imports and files** 

In [ ]:
import numpy as np
from sklearn.model_selection import KFold
import matplotlib.pyplot as plt
import time
import pickle
import torch
import torchvision
from torch import nn
from torch.nn import functional as F
from torch.utils.data import Dataset
from torch.utils.data import DataLoader, random_split
from torchvision.datasets import CIFAR10
from torchvision import transforms
from torch.optim.lr_scheduler import OneCycleLR
from torchvision.utils import make_grid
from torch.utils.data import RandomSampler
import torchvision.models as models
!pip install lightning
!pip install neptune
import lightning as L
import neptune 
from torchmetrics import Accuracy

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# !cp "/content/drive/My Drive/Deep_learning_task1/alex.ckpt" .
# !cp "/content/drive/My Drive/Deep_learning_task1/vgg.ckpt" .
# !cp "/content/drive/My Drive/Deep_learning_task1/efficient.ckpt" .
# !cp "/content/drive/My Drive/Deep_learning_task1/google.ckpt" .
!cp "/content/drive/My Drive/Deep_learning_task1/kaggle.json" .

In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cpu')

In [4]:
import os, sys, random, shutil
from google.colab import files

if 'google.colab' in str(get_ipython()):
    files.upload()
    os.system('mkdir ~/.kaggle')
    os.system('mv ./kaggle.json ~/.kaggle/')
    os.system('chmod 600 ~/.kaggle/kaggle.json')
    os.system('kaggle datasets download -d gpiosenka/100-bird-species')
    os.system('unzip 100-bird-species.zip')
    root_address = '.'
else:
    root_address = '../input/100-bird-species'

# **Dataset into data loaders**

In [5]:
from torchvision.datasets import ImageFolder
from torchvision.transforms import ToTensor,Lambda
from torch.utils.data import SubsetRandomSampler
from torchvision.transforms import transforms, RandomCrop, RandomHorizontalFlip, RandomRotation

BATCH_SIZE = 32

train_path = './train'
val_path = './valid'
test_path = './test'
transform = transforms.Compose([transforms.Resize((224, 224)), 
                                transforms.ToTensor()])


train_data = ImageFolder(root = train_path,transform=transform)
val_data = ImageFolder(root = val_path,transform=transform)
test_data = ImageFolder(root = test_path,transform=transform)

train_random_sampler = RandomSampler(train_data)

valid_random_sampler = RandomSampler(val_data)

test_random_sampler = RandomSampler(test_data)

train_loader = DataLoader(
    dataset = train_data,
    batch_size = BATCH_SIZE,
    sampler = train_random_sampler,
)

val_loader = DataLoader(
    dataset = val_data,
    batch_size = BATCH_SIZE,
    sampler = valid_random_sampler,
)

test_loader = DataLoader(
    dataset = test_data,
    batch_size = BATCH_SIZE,
    sampler = test_random_sampler,
)

# **data analasys**

In [7]:
classes = os.listdir(train_path)
print("Total Classes: ",len(classes))
print("Shape of an individual image: ", train_data[0][0].shape)

train_count = 0
valid_count = 0
test_count = 0
for _class in classes:
    train_count += len(os.listdir('./train/' + _class))
    valid_count += len(os.listdir('./valid/' + _class))
    test_count += len(os.listdir('./test/' + _class))

classes = dict()
directory_paths = ['./train', './valid', './test']

for path in directory_paths:
  for folder_name in os.listdir(path):
      folder_path = os.path.join(path, folder_name)
      
      if os.path.isdir(folder_path):
          file_count = len(os.listdir(folder_path))       
          if folder_name not in classes: 
            classes[folder_name] = file_count
          else:
            classes[folder_name] += file_count

min = ("",np.Inf)
max = ("",0)
avg = 0

for key,value in classes.items():
  if value > max[1]:
    max = (key, value)
  if value < min[1]:
    min = (key, value)
  avg += value

print("Total train images: ",train_count)
print("Total valid images: ",valid_count)
print("Total test images: ",test_count)


print("Max samples: ", max)
print("Min samples: ", min)
print("Avg samples: ", avg/510)

Total Classes:  515
Shape of an individual image:  torch.Size([3, 224, 224])
Total train images:  82724
Total valid images:  2575
Total test images:  2575
Max samples:  ('RUFOUS TREPE', 273)
Min samples:  ('EURASIAN GOLDEN ORIOLE', 140)
Avg samples:  172.30196078431374


# **Transfer learning class**

In [ ]:
class birdNet(L.LightningModule):
  def __init__(self):
        super().__init__()

  def __name__(self):
      return 'birdNet'

  def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(),lr=self.lr)

  def forward(self, x):
        return self.model(x)

  def training_step(self, batch, batch_idx):
        images, labels = batch
        images = images.to(self.device)
        labels = labels.to(self.device)
        
        preds = self.model(images)                
        loss = self.criterion(preds, labels)

        run[f"train_/loss"].append(loss)
        return loss

  def validation_step(self, batch, batch_idx):
        images, labels = batch
        images = images.to(self.device)
        labels = labels.to(self.device)
        
        preds = self.model(images)    
        accuracy=self.val_accuracy(preds, labels).item()
        loss = self.criterion(preds, labels)

        self.run['validation/loss'].append(loss)
        self.run['validation/accuracy'].append(accuracy)
        self.log("val_loss", loss, prog_bar=True)
        self.log("val_acc", accuracy , prog_bar=True,on_step=True)

        return loss

  def test_step(self, batch, batch_idx):
      images, labels = batch
      images = images.to(self.device)
      labels = labels.to(self.device)
      
      preds = self.model(images)        
      accuracy=self.val_accuracy(preds, labels).item()
      loss = self.criterion(preds, labels)

      self.run['test/loss'].append(loss)
      self.run['test/accuracy'].append(accuracy)
      self.log("test_loss", loss, prog_bar=True)
      self.log("test_acc", accuracy, prog_bar=True,on_step=True)

      return loss
  
  def unfreeze(self):
    for param in self.model.parameters():
      param.requires_grad = True 

  def changeLR(self, value):
    self.lr = value
  
class efficientBirdNet(birdNet):
    def __init__(self):
        super().__init__()        
        self.model = models.efficientnet_b0(pretrained = True)
        for param in self.model.parameters():
            param.requires_grad = False  
        # self.model.classifier = nn.Sequential(nn.Linear(1280, 515))
        self.model.classifier = nn.Sequential(
            nn.Dropout(0.5),
            nn.Linear(1280, 1280),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(1280, 515)
        )
        self.model.to(device)
        
        self.criterion = nn.CrossEntropyLoss()
        self.lr = 0.0001
        self.run = run

        self.val_accuracy = Accuracy(task="multiclass", num_classes=515)
        self.test_accuracy = Accuracy(task="multiclass", num_classes=515)
class VGGBirdNet(birdNet): 
    def __init__(self):
        super().__init__()        
        self.model = models.vgg16(pretrained = True)
        for param in self.model.parameters():
            param.requires_grad = False 
        # self.model.classifier = nn.Sequential(nn.Linear(25088, 515))
        self.model.classifier = nn.Sequential(
            nn.Dropout(0.5),
            nn.Linear(25088, 1280),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(1280, 515)
        )
        self.model.to(device)
        
        self.criterion = nn.CrossEntropyLoss()
        self.lr = 0.0001
        self.run = run

        self.val_accuracy = Accuracy(task="multiclass", num_classes=515)
        self.test_accuracy = Accuracy(task="multiclass", num_classes=515)
class alexBirdNet(birdNet): 
    def __init__(self):
        super().__init__()        
        self.model = models.alexnet(pretrained = True)
        for param in self.model.parameters():
            param.requires_grad = False 
        # self.model.classifier = nn.Sequential(nn.Linear(9216, 515))
        self.model.classifier = nn.Sequential(
            nn.Dropout(0.5),
            nn.Linear(9216, 1280),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(1280, 515)
        )
        self.model.to(device)
        
        self.criterion = nn.CrossEntropyLoss()
        self.lr = 0.0001
        self.run = run

        self.val_accuracy = Accuracy(task="multiclass", num_classes=515)
        self.test_accuracy = Accuracy(task="multiclass", num_classes=515)
class googleBirdNet(birdNet): 
    def __init__(self):
        super().__init__()        
        self.model = models.googlenet(pretrained = True)
        for param in self.model.parameters():
            param.requires_grad = False 
        # self.model.fc = nn.Sequential(nn.Linear(1024, 515))
        self.model.fc = nn.Sequential(
            nn.Dropout(0.5),
            nn.Linear(1024, 1024),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(1024, 515)
        )
        self.model.to(device)
        
        self.criterion = nn.CrossEntropyLoss()
        self.lr = 0.0001
        self.run = run

        self.val_accuracy = Accuracy(task="multiclass", num_classes=515)
        self.test_accuracy = Accuracy(task="multiclass", num_classes=515)

In [ ]:
run = neptune.init_run(
    project="sadotal/ML-task1",
    api_token="eyJhcGlfYWRkcmVzcyI6Imh0dHBzOi8vbmV3LXVpLm5lcHR1bmUuYWkiLCJhcGlfdXJsIjoiaHR0cHM6Ly9uZXctdWkubmVwdHVuZS5haSIsImFwaV9rZXkiOiJmMjIwNjBjMS1kMDhhLTQ4MzctYmIwZS05ZDUyY2E2ZjZiM2QifQ==",
)

<ipython-input-9-262b5f2a57a1>:1: NeptuneWarning: To avoid unintended consumption of logging hours during interactive sessions, the following monitoring options are disabled unless set to 'True' when initializing the run: 'capture_stdout', 'capture_stderr', and 'capture_hardware_metrics'.
  run = neptune.init_run(


https://new-ui.neptune.ai/sadotal/ML-task1/e/MLTAS-110


# **efficientnet_b0**

In [ ]:
efficientnet_trainer = L.Trainer(
    accelerator='auto',
    default_root_dir="efficientnet/",
    max_epochs=1
)

run["sys/tags"].add('Pretrained Model: Efficientnet_B0 11 epochs, using added layers and fine tunning')
run["config/lr"]=0.0001
run["config/batch_size"]=BATCH_SIZE
efficientnet = efficientBirdNet()
efficientnet_trainer.fit(efficientnet, train_loader, val_loader)

efficientnet.unfreeze()
efficientnet.changeLR(0.00001)

efficientnet_trainer = L.Trainer(
    accelerator='auto',
    default_root_dir="efficientnet/",
    max_epochs=10  
)
efficientnet_trainer.fit(efficientnet, train_loader, val_loader)

In [ ]:
print('===vaidate===')
efficientnet_trainer.validate(efficientnet,val_loader)
print('===test===')
efficientnet_trainer.test(efficientnet,test_loader)

INFO: You are using a CUDA device ('NVIDIA A100-SXM4-40GB') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
INFO:lightning.pytorch.utilities.rank_zero:You are using a CUDA device ('NVIDIA A100-SXM4-40GB') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
INFO: LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:lightning.pytorch.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


===vaidate===


Validation: 0it [00:00, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃      Validate metric      ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│       val_acc_epoch       │     0.976310670375824     │
│         val_loss          │    0.11658814549446106    │
└───────────────────────────┴───────────────────────────┘

INFO: You are using a CUDA device ('NVIDIA A100-SXM4-40GB') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
INFO:lightning.pytorch.utilities.rank_zero:You are using a CUDA device ('NVIDIA A100-SXM4-40GB') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
INFO: LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:lightning.pytorch.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


===test===


/usr/local/lib/python3.9/dist-packages/lightning/pytorch/trainer/connectors/data_connector.py:478: PossibleUserWarning: Your `test_dataloader`'s sampler has shuffling enabled, it is strongly recommended that you turn shuffling off for val/test dataloaders.
  rank_zero_warn(
/usr/local/lib/python3.9/dist-packages/lightning/pytorch/trainer/connectors/data_connector.py:430: PossibleUserWarning: The dataloader, test_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 12 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Testing: 0it [00:00, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│      test_acc_epoch       │    0.9887378811836243     │
│         test_loss         │    0.0396064817905426     │
└───────────────────────────┴───────────────────────────┘

[{'test_loss': 0.0396064817905426, 'test_acc_epoch': 0.9887378811836243}]

# **VGG16**

In [ ]:
vgg_trainer = L.Trainer(
    accelerator='auto',
    default_root_dir="VGG16/",
    max_epochs=1
)

run["sys/tags"].add('Pretrained Model: VGG16 11 epochs, using added layers and fine tunning')
run["config/lr"]=0.0001
run["config/batch_size"]=BATCH_SIZE
vgg = VGGBirdNet()
vgg_trainer.fit(vgg, train_loader, val_loader)

vgg.unfreeze()
vgg.changeLR(0.00001)

vgg_trainer = L.Trainer(
    accelerator='auto',
    default_root_dir="VGG16/",
    max_epochs=10 
)
vgg_trainer.fit(vgg, train_loader, val_loader)

INFO: GPU available: True (cuda), used: True
INFO:lightning.pytorch.utilities.rank_zero:GPU available: True (cuda), used: True
INFO: TPU available: False, using: 0 TPU cores
INFO:lightning.pytorch.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO: IPU available: False, using: 0 IPUs
INFO:lightning.pytorch.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO: HPU available: False, using: 0 HPUs
INFO:lightning.pytorch.utilities.rank_zero:HPU available: False, using: 0 HPUs
/usr/local/lib/python3.9/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
INFO: You are using a CUDA device ('NVIDIA A100-SXM4-40GB') that has Tensor Cores. To properly utilize t

Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

INFO: `Trainer.fit` stopped: `max_epochs=1` reached.
INFO:lightning.pytorch.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=1` reached.
INFO: GPU available: True (cuda), used: True
INFO:lightning.pytorch.utilities.rank_zero:GPU available: True (cuda), used: True
INFO: TPU available: False, using: 0 TPU cores
INFO:lightning.pytorch.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO: IPU available: False, using: 0 IPUs
INFO:lightning.pytorch.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO: HPU available: False, using: 0 HPUs
INFO:lightning.pytorch.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO: You are using a CUDA device ('NVIDIA A100-SXM4-40GB') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul

...

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

INFO: `Trainer.fit` stopped: `max_epochs=10` reached.
INFO:lightning.pytorch.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=10` reached.


In [ ]:
print('===vaidate===')
vgg_trainer.validate(vgg,val_loader)
print('===test===')
vgg_trainer.test(vgg,test_loader)

INFO: You are using a CUDA device ('NVIDIA A100-SXM4-40GB') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
INFO:lightning.pytorch.utilities.rank_zero:You are using a CUDA device ('NVIDIA A100-SXM4-40GB') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
INFO: LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:lightning.pytorch.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


===vaidate===


Validation: 0it [00:00, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃      Validate metric      ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│       val_acc_epoch       │    0.9409708976745605     │
│         val_loss          │    0.2496330738067627     │
└───────────────────────────┴───────────────────────────┘

INFO: You are using a CUDA device ('NVIDIA A100-SXM4-40GB') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
INFO:lightning.pytorch.utilities.rank_zero:You are using a CUDA device ('NVIDIA A100-SXM4-40GB') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
INFO: LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:lightning.pytorch.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


===test===


Testing: 0it [00:00, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│      test_acc_epoch       │    0.9596116542816162     │
│         test_loss         │    0.13597215712070465    │
└───────────────────────────┴───────────────────────────┘

[{'test_loss': 0.13597215712070465, 'test_acc_epoch': 0.9596116542816162}]

# **AlexNet**

In [ ]:
alex_trainer = L.Trainer(
    accelerator='auto',
    default_root_dir="alex/",
    max_epochs=1
)

run["sys/tags"].add('Pretrained Model: alexNet 11 epochs, using added layers and fine tunning')
run["config/lr"]=0.0001
run["config/batch_size"]=BATCH_SIZE
alex = alexBirdNet()
alex_trainer.fit(alex, train_loader, val_loader)

alex.unfreeze()
alex.changeLR(0.00001)

alex_trainer = L.Trainer(
    accelerator='auto',
    default_root_dir="alex/",
    max_epochs=10 
)
alex_trainer.fit(alex, train_loader, val_loader)

INFO: GPU available: True (cuda), used: True
INFO:lightning.pytorch.utilities.rank_zero:GPU available: True (cuda), used: True
INFO: TPU available: False, using: 0 TPU cores
INFO:lightning.pytorch.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO: IPU available: False, using: 0 IPUs
INFO:lightning.pytorch.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO: HPU available: False, using: 0 HPUs
INFO:lightning.pytorch.utilities.rank_zero:HPU available: False, using: 0 HPUs
/usr/local/lib/python3.9/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=AlexNet_Weights.IMAGENET1K_V1`. You can also use `weights=AlexNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/alexnet-owt-7be5be79.pth" to /root/.cache/torch/hu

Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

INFO: `Trainer.fit` stopped: `max_epochs=1` reached.
INFO:lightning.pytorch.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=1` reached.
INFO: GPU available: True (cuda), used: True
INFO:lightning.pytorch.utilities.rank_zero:GPU available: True (cuda), used: True
INFO: TPU available: False, using: 0 TPU cores
INFO:lightning.pytorch.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO: IPU available: False, using: 0 IPUs
INFO:lightning.pytorch.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO: HPU available: False, using: 0 HPUs
INFO:lightning.pytorch.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO: You are using a CUDA device ('NVIDIA A100-SXM4-40GB') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul

...

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

INFO: `Trainer.fit` stopped: `max_epochs=10` reached.
INFO:lightning.pytorch.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=10` reached.


In [ ]:
print('===vaidate===')
alex_trainer.validate(alex,val_loader)
print('===test===')
alex_trainer.test(alex,test_loader)

INFO: You are using a CUDA device ('NVIDIA A100-SXM4-40GB') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
INFO:lightning.pytorch.utilities.rank_zero:You are using a CUDA device ('NVIDIA A100-SXM4-40GB') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
INFO: LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:lightning.pytorch.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


===vaidate===


Validation: 0it [00:00, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃      Validate metric      ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│       val_acc_epoch       │    0.9149514436721802     │
│         val_loss          │    0.3464323878288269     │
└───────────────────────────┴───────────────────────────┘

INFO: You are using a CUDA device ('NVIDIA A100-SXM4-40GB') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
INFO:lightning.pytorch.utilities.rank_zero:You are using a CUDA device ('NVIDIA A100-SXM4-40GB') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
INFO: LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:lightning.pytorch.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


===test===


Testing: 0it [00:00, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│      test_acc_epoch       │    0.9374757409095764     │
│         test_loss         │    0.2014760971069336     │
└───────────────────────────┴───────────────────────────┘

[{'test_loss': 0.2014760971069336, 'test_acc_epoch': 0.9374757409095764}]

# **GoogLeNet**

In [ ]:
google_trainer = L.Trainer(
    accelerator='auto',
    default_root_dir="google/",
    max_epochs=1
)

run["sys/tags"].add('Pretrained Model: google 11 epochs, using added layers and fine tunning')
run["config/lr"]=0.0001
run["config/batch_size"]=BATCH_SIZE
google = googleBirdNet()
google_trainer.fit(google, train_loader, val_loader)

google.unfreeze()
google.changeLR(0.00001)

google_trainer = L.Trainer(
    accelerator='auto',
    default_root_dir="google/",
    max_epochs=10 
)
google_trainer.fit(google, train_loader, val_loader)

In [ ]:
print('===vaidate===')
google_trainer.validate(google,val_loader)
print('===test===')
google_trainer.test(google,test_loader)

INFO: You are using a CUDA device ('NVIDIA A100-SXM4-40GB') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
INFO:lightning.pytorch.utilities.rank_zero:You are using a CUDA device ('NVIDIA A100-SXM4-40GB') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
INFO: LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:lightning.pytorch.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


===vaidate===


Validation: 0it [00:00, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃      Validate metric      ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│       val_acc_epoch       │    0.9537864327430725     │
│         val_loss          │     0.163270965218544     │
└───────────────────────────┴───────────────────────────┘

INFO: You are using a CUDA device ('NVIDIA A100-SXM4-40GB') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
INFO:lightning.pytorch.utilities.rank_zero:You are using a CUDA device ('NVIDIA A100-SXM4-40GB') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
INFO: LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:lightning.pytorch.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


===test===


/usr/local/lib/python3.9/dist-packages/lightning/pytorch/trainer/connectors/data_connector.py:478: PossibleUserWarning: Your `test_dataloader`'s sampler has shuffling enabled, it is strongly recommended that you turn shuffling off for val/test dataloaders.
  rank_zero_warn(
/usr/local/lib/python3.9/dist-packages/lightning/pytorch/trainer/connectors/data_connector.py:430: PossibleUserWarning: The dataloader, test_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 12 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Testing: 0it [00:00, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│      test_acc_epoch       │     0.976310670375824     │
│         test_loss         │    0.0882675051689148     │
└───────────────────────────┴───────────────────────────┘

[{'test_loss': 0.0882675051689148, 'test_acc_epoch': 0.976310670375824}]

# **Calculate unique errors and correct classification**

In [ ]:
model_list = [efficientnet, vgg, alex, google]
model_names = ['efficientnet', 'vgg', 'alexnet', 'googlenet']
num_models = len(model_list)

unique_correct_counts = [{}, {}, {}, {}]
unique_error_counts = [{}, {}, {}, {}]

for images, labels in test_loader:
    images = images.to(device)
    label = labels.tolist()[0]
    
    predicted_labels = []
    
    for i in range(num_models):
        model_list[i] = model_list[i].to(device)
        outputs = model_list[i](images)
        predicted = torch.argmax(outputs, dim=1).tolist()[0]
        predicted_labels.append(predicted)
    
    for i in range(num_models):
        if predicted_labels[i] == label:
            if all(predicted_labels[j] != label for j in range(num_models) if j != i):
                if label in unique_correct_counts[i]:
                    unique_correct_counts[i][label] += 1
                else:
                    unique_correct_counts[i][label] = 1
        else:
            if all(predicted_labels[j] == label for j in range(num_models) if j != i):
                if label in unique_error_counts[i]:
                    unique_error_counts[i][label] += 1
                else:
                    unique_error_counts[i][label] = 1

for i in range(num_models):
    print(f"Total unique correct samples for {model_names[i]}: {sum(unique_correct_counts[i].values())}")
    print(f"Total unique error samples for {model_names[i]}: {sum(unique_error_counts[i].values())}")

Total unique correct samples for efficientnet: 1
Total unique error samples for efficientnet: 0
Total unique correct samples for vgg: 1
Total unique error samples for vgg: 3
Total unique correct samples for alexnet: 0
Total unique error samples for alexnet: 7
Total unique correct samples for googlenet: 1
Total unique error samples for googlenet: 1


# **Using the trained model as a feature extractor for random forest classifier**

In [ ]:
from sklearn.ensemble import RandomForestClassifier

class PreTrainedFeatureClassifier():
  def __init__(self, model, train_loader, test_loader, device):
    self.model = torch.nn.Sequential(*(list(model.model.children())[:-1])).to(device) # Removing the last layer of the model
    self.train_loader = train_loader
    self.test_loader = test_loader

    self.train_features = []
    self.train_labels = []

    self.test_features = []
    self.test_labels = []
    
    self.classifier = None
    self.device = device

  def extractFeatures(self):
      train_features = []
      train_labels = []

      test_features = []
      test_labels = []

      with torch.no_grad():
          self.model.eval()

          for x, y in self.train_loader:
              x = x.to(self.device)
              feature = self.model(x)
              feature_np = feature.detach().cpu().numpy().reshape(x.shape[0], -1)
              train_features.append(feature_np)
              train_labels.append(y.cpu().numpy())

          for x, y in self.test_loader:
              x = x.to(self.device)            
              feature = self.model(x)
              feature_np = feature.detach().cpu().numpy().reshape(x.shape[0], -1)
              test_features.append(feature_np)
              test_labels.append(y.cpu().numpy())

      self.train_features = np.concatenate(train_features, axis=0)
      self.train_labels = np.concatenate(train_labels, axis=0)
      self.test_features = np.concatenate(test_features, axis=0)
      self.test_labels = np.concatenate(test_labels, axis=0)
  
  def fit(self):
    self.classifier = RandomForestClassifier(n_estimators=50, max_depth=150, random_state=42)
    self.classifier.fit(self.train_features, self.train_labels)
  
  def evaluate(self):
    return self.classifier.score(self.test_features, self.test_labels)
  

In [ ]:
forest_classifier = PreTrainedFeatureClassifier(efficientnet, train_loader, test_loader, device)
forest_classifier.extractFeatures()
forest_classifier.fit()

In [ ]:
forest_classifier.evaluate()

0.9666019417475729